# SpaceX Dashboard

In [1]:
import pandas as pd
import numpy as np
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt
import seaborn as sns
print('Imported Libraries')

Imported Libraries


In [2]:
#Create app
app = dash.Dash(__name__)
print('Created Dash App')

Created Dash App


In [3]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [5]:
df.shape

(56, 7)

In [6]:
df['Launch Site'].unique()
df = df.assign(Outcome = df['class'].apply(lambda x: True if 1 else False))

## TASK 1: Add a Launch Site Drop-down Input Component

## TASK 3: Add a Range Slider to Select Payload

In [7]:
range_markers = dict(zip([i for i in range(0, 10001, 1000)], [str(i) for i in range(0, 20001, 1000)]))

# Layout Section of Dash
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard', 
                                style={'textAlign': 'center', 'color': '#6b0303', 'font-size': 36}),
    html.Div([
        
            # Launch Site Dropdown List
            html.Div([
                html.H2('Select Site:', style={'margin-right': '2em'}),
                dcc.Dropdown(id='select-site',
                             options=[{'label': i, 'value': i} for i in df['Launch Site'].unique()], 
                             value='Select Site',
                             placeholder='All Sites', 
                             disabled=False)
            ]),
        
            # Pie Chart Display
            html.Div(id='pie-container', className='chart-grid'),

            # Payload Range Slider
            html.Div([
                html.H2('Select Payload:', style={'margin-right': '2em'}),        
                dcc.RangeSlider(id='select-payload',
                                min=0, max=10000, step=1000,
                                marks = range_markers,
                                value=[1000,9000],
                                disabled=False)
            ]),
        
            # Scatter Chart Display
            html.Div(id='scatter-container', className='chart-grid'),      
        ])
    ])

## TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

In [8]:
@app.callback( Output(component_id='pie-container', component_property='children'),
               Input(component_id='select-site', component_property='value'))

def display_charts(input_site):

    # Pie Chart Launch Success by Site
    data = df[["class", 'Launch Site']]
    if input_site == None:
        data = data.groupby('Launch Site')['class'].sum().reset_index()
        pie_fig = px.pie(data, values='class', names='Launch Site')
        pie_fig.update_layout(title=f'Total Successes Launch by Site')
    else:
        data = df[df['Launch Site']==input_site]
        data = data.groupby(['class'])['class'].count().reset_index(name='count')
        data = data.assign(Outcome = data['class'].apply(lambda x: 'Success' if x==1 else 'Failure'))
        pie_fig = px.pie(data, values='count', names='Outcome')
        pie_fig.update_layout(title=f'Total Successes Launch for Site  {input_site}')
        pie_fig.update_layout(height=400, width=1000)
    
    return [html.Div(className='chart-item', children=[dcc.Graph(figure=pie_fig)], style={'display': 'flex'})]


## TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

In [9]:
@app.callback( Output(component_id='scatter-container', component_property='children'),
               Input(component_id='select-payload', component_property='value'))

def display_charts(input_payload):

    # Scatter Plot Launch Success by Payload
    min_payload = input_payload[0]
    max_payload = input_payload[1]
    data = df[["Payload Mass (kg)", "class", 'Booster Version Category']]
    data = data[data['Payload Mass (kg)'] >= min_payload]
    data = data[data['Payload Mass (kg)'] <= max_payload]
    scatter_fig = px.scatter(data, x='Payload Mass (kg)', y='class', color='Booster Version Category', color_discrete_sequence=['red', 'blue', 'orange', 'green', 'purple']) 
    scatter_fig.update_layout(title="Correlation between Payload and Success for all Sites", xaxis_title='Payload', yaxis_title=f'Success')
    scatter_fig.update_traces(marker=dict(size=12))
    scatter_fig.update_layout(height=400, width=1200)
    
    return [html.Div(className='chart-item', children=[dcc.Graph(figure=scatter_fig)], style={'display': 'flex'})]

In [10]:
## Creates Dashboard at http://localhost:{port}/
if __name__ == '__main__':
    app.run_server(port=8050,
                   mode='jupyterlab',
                   dev_tools_ui=True,
                   dev_tools_hot_reload=True, 
                   threaded=True)